In [101]:
import argparse
import logging
import sys
import time
import warnings

import numpy as np
import pandas as pd
import torch
from scipy.stats import pearsonr
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (average_precision_score,
                             classification_report, mean_squared_error, r2_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import  nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA

import sampling as sam
#import utils as ut
import trainers as t
from models import (AEBase,PretrainedPredictor, PretrainedVAEPredictor, VAEBase)
import matplotlib

In [7]:
import utils as ut

In [10]:
import os
os.getcwd()

'/Users/stefano.cardinale/scDEAL/scDEAL'

In [92]:
epochs = 500
dim_au_out = 256
select_drug = 'I.BET.762'
na = 1
data_path='data/GDSC2_expression.csv'
label_path='data/GDSC2_label_192drugs_binary.csv'
test_size = 0.2
valid_size = 0.2
g_disperson = None
model_path = 'saved/models/bulk_predictor_'
bulk_encoder = 'saved/models/bulk_encoder_ae_256_test.pkl' #This are the weights for the pretrained AE model
log_path = 'saved/logs/log'
batch_size = 200
encoder_hdims = "256,256".split(",")
preditor_hdims = "128,64".split(",")
reduce_model = "AE"
sampling = None
PCA_dim = 0
pretrain = 1
freeze_pretrain = 0
VAErepram = 1

encoder_hdims = list(map(int, encoder_hdims) )
preditor_hdims = list(map(int, preditor_hdims) )
load_model = bool(0)

preditor_path = model_path + reduce_model  + select_drug + '.pkl'

In [105]:
# Initialize logging and std out
now=time.strftime("%Y-%m-%d-%H-%M-%S")

#ut.save_arguments(now)

out_path = log_path+now+".err"
log_path = log_path+now+".log"

out=open('./debug.log',"w")
#sys.stderr=out
logging.basicConfig(filename='./debug.log', encoding='utf-8', level=logging.DEBUG)


In [17]:
# Read data

data_r=pd.read_csv(data_path,index_col=0)
label_r=pd.read_csv(label_path,index_col=0)
label_r=label_r.fillna(na)


now=time.strftime("%Y-%m-%d-%H-%M-%S")

selected_idx = label_r.loc[:,select_drug]!=na

In [20]:
if(g_disperson!=None):
    hvg,adata = ut.highly_variable_genes(data_r,min_disp=g_disperson)
    # Rename columns if duplication exist
    data_r.columns = adata.var_names
    # Extract hvgs
    data = data_r.loc[selected_idx.index,hvg]
else:
    data = data_r.loc[selected_idx.index,:]

# Do PCA if PCA_dim!=0
if PCA_dim !=0 :
    data = PCA(n_components = PCA_dim).fit_transform(data)
else:
    data = data

In [22]:
# Extract labels
label = label_r.loc[selected_idx.index,select_drug]
data_r = data_r.loc[selected_idx.index,:]

# Scaling data
mmscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = label.values.reshape(-1,1)


le = LabelEncoder()
label = le.fit_transform(label)
dim_model_out = 2

/Users/stefano.cardinale/opt/anaconda3/envs/scDEAL/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [106]:
logging.info(np.std(data))
logging.info(np.mean(data))

In [96]:
# Split traning valid test set
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=test_size, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=valid_size, random_state=42)
# sampling method
if sampling == None:
    X_train,Y_train=sam.nosampling(X_train,Y_train)
    logging.info("nosampling")
elif sampling =="upsampling":
    X_train,Y_train=sam.upsampling(X_train,Y_train)
    logging.info("upsampling")
elif sampling =="downsampling":
    X_train,Y_train=sam.downsampling(X_train,Y_train)
    logging.info("downsampling")
elif  sampling=="SMOTE":
    X_train,Y_train=sam.SMOTEsampling(X_train,Y_train)
    logging.info("SMOTE")
else:
    logging.info("not a legal sampling method")

In [72]:
from copy import deepcopy
h_dims=[512]
hidden_dims = deepcopy(h_dims)
hidden_dims.insert(0,515)
print(hidden_dims)

hidden_dims.reverse()
print(hidden_dims)

[515, 512]
[512, 515]


In [78]:
# Select the Training device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
logging.info(device)
#torch.cuda.set_device(device)

# Construct datasets and data loaders
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)

Y_trainTensor = torch.LongTensor(Y_train).to(device)
Y_validTensor = torch.LongTensor(Y_valid).to(device)

# Preprocess data to tensor
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=batch_size, shuffle=True)
validDataLoader_p = DataLoader(dataset=validreducedDataset, batch_size=batch_size, shuffle=True)

dataloaders_train = {'train':trainDataLoader_p,'val':validDataLoader_p}

if(bool(pretrain)!=False):
    dataloaders_pretrain = {'train':X_trainDataLoader,'val':X_validDataLoader}
    if reduce_model == "VAE":
        encoder = VAEBase(input_dim=data.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims)
    else:
        encoder = AEBase(input_dim=data.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims)

    if torch.cuda.is_available():
        encoder.cuda()

    logging.info(encoder)
    encoder.to(device)

    optimizer_e = optim.Adam(encoder.parameters(), lr=1e-2)
    loss_function_e = nn.MSELoss()
    exp_lr_scheduler_e = lr_scheduler.ReduceLROnPlateau(optimizer_e)

    if reduce_model == "AE":
        encoder,loss_report_en = t.train_AE_model(net=encoder,data_loaders=dataloaders_pretrain,
                                    optimizer=optimizer_e,loss_function=loss_function_e,
                                    n_epochs=epochs,scheduler=exp_lr_scheduler_e,save_path=bulk_encoder)
    elif reduce_model == "VAE":
        encoder,loss_report_en = t.train_VAE_model(net=encoder,data_loaders=dataloaders_pretrain,
                        optimizer=optimizer_e,
                        n_epochs=epochs,scheduler=exp_lr_scheduler_e,save_path=bulk_encoder)

KeyboardInterrupt: 

In [41]:
if reduce_model == "AE":
        model = PretrainedPredictor(input_dim=X_train.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims, 
                                hidden_dims_predictor=preditor_hdims,output_dim=dim_model_out,
                                pretrained_weights=bulk_encoder,freezed=bool(freeze_pretrain))
elif reduce_model == "VAE":
    model = PretrainedVAEPredictor(input_dim=X_train.shape[1],latent_dim=dim_au_out,h_dims=encoder_hdims, 
                    hidden_dims_predictor=preditor_hdims,output_dim=dim_model_out,
                    pretrained_weights=bulk_encoder,freezed=bool(freeze_pretrain),z_reparam=bool(VAErepram))

logging.info("Current model is:")
logging.info(model)
if torch.cuda.is_available():
    model.cuda()
    
model.to(device)

PretrainedPredictor(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=17419, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=256, out_features=256, bias=True)
  (predictor): Predictor(
    (predictor): Sequential(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.3, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1

In [42]:
# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-2)


loss_function = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

# Train prediction model
model,report = t.train_predictor_model(model,dataloaders_train,
                                        optimizer,loss_function,epochs,exp_lr_scheduler,load=load_model,save_path=preditor_path)
dl_result = model(X_testTensor).detach().cpu().numpy()


In [44]:
lb_results = np.argmax(dl_result,axis=1)
#pb_results = np.max(dl_result,axis=1)
pb_results = dl_result[:,1]

report_dict = classification_report(Y_test, lb_results, output_dict=True)
report_df = pd.DataFrame(report_dict).T
ap_score = average_precision_score(Y_test, pb_results)
auroc_score = roc_auc_score(Y_test, pb_results)

report_df['auroc_score'] = auroc_score
report_df['ap_score'] = ap_score

report_df.to_csv("saved/logs/" + reduce_model + select_drug+now + '_report.csv')

logging.info(classification_report(Y_test, lb_results))
logging.info(average_precision_score(Y_test, pb_results))
logging.info(roc_auc_score(Y_test, pb_results))

model = DummyClassifier(strategy='stratified')
model.fit(X_train, Y_train)
yhat = model.predict_proba(X_test)
naive_probs = yhat[:, 1]

ut.plot_roc_curve(Y_test, naive_probs, pb_results, title=str(roc_auc_score(Y_test, pb_results)),
                    path="saved/figures/" + reduce_model + select_drug+now + '_roc.pdf')
ut.plot_pr_curve(Y_test,pb_results,  title=average_precision_score(Y_test, pb_results),
                path="saved/figures/" + reduce_model + select_drug+now + '_prc.pdf')